In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from utils import *

import os
print(os.listdir("../input/mini-model-data/"))

['test_df_h.pkl', 'test_df_c.pkl', 'train_df_w.pkl', 'test_df_w.pkl', 'train_df_s.pkl', 'test_df_s.pkl', 'test_df_r.pkl', 'train_df_h.pkl', 'test_identity.pkl', 'train_df_r.pkl', 'train_identity.pkl', 'train_df_c.pkl']


In [2]:
train = pd.read_pickle('../input/mini-model-features/train_features_s.pkl')
test= pd.read_pickle('../input/mini-model-features/test_features_s.pkl')

train_y = train['isFraud'].copy()

print(train.shape)
print(train_y.shape)
print(test.shape)

(11628, 441)
(11628,)
(11418, 441)


In [3]:
rm_cols = [
    'isFraud', 'TransactionDT', 'DT', 'addr2', 'C6', 'TransactionAmt', 'id_18'
    'id_21', 'id_22','id_30', 'id_31', 'id_34',
   'V28', 'V65','V88',
    'DeviceInfo', 
]
features_columns = list(train)
for col in rm_cols:
    if col in features_columns:
        features_columns.remove(col)

In [7]:
train[features_columns].head()

,D1,D2,D3,D6,D7,D10,D13,D14,D15,V14,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V31,V32,V55,V56,V59,V60,V61,V62,V63,V64,V66,V67,V68,V71,V72,V77,V78,V80,V81,V84,V85,V86,V87,V89,V92,V93,V95,V96,V97,V99,V100,V101,V102,V103,V104,V105,V106,V108,V109,V110,V111,V114,V115,V116,V120,V123,V124,V125,V126,V127,V128,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V160,V162,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V207,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V223,V224,V225,V226,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V240,V242,V243,V244,V246,V247,V248,V249,V252,V253,V254,V257,V258,V260,V261,V262,V263,V264,V265,V266,V268,V269,V270,V271,V272,V273,V274,V275,V279,V280,V281,V282,V283,V285,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V306,V307,V308,V310,V312,V313,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V326,V327,V328,V329,V330,V331,V332,V333,V335,V336,V337,V338,V339,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,card3_TransactionAmt_mean,uid_TransactionAmt_mean,uid_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,uid3_TransactionAmt_std,D3_isnull,D6_isnull,D7_isnull,D8_isnull,D9_isnull,D10_isnull,D13_isnull,D14_isnull,D15_isnull,V14_isnull,V17_isnull,V18_isnull,V19_isnull,V20_isnull,V21_isnull,V22_isnull,V23_isnull,V24_isnull,V25_isnull,V26_isnull,V27_isnull,V28_isnull,V31_isnull,V32_isnull,V55_isnull,V56_isnull,V59_isnull,V60_isnull,V61_isnull,V62_isnull,V63_isnull,V64_isnull,V65_isnull,V66_isnull,V67_isnull,V68_isnull,V71_isnull,V72_isnull,V77_isnull,V78_isnull,V80_isnull,V81_isnull,V84_isnull,V85_isnull,V86_isnull,V87_isnull,V88_isnull,V89_isnull,V92_isnull,V93_isnull,id_03_isnull,id_04_isnull,id_09_isnull,id_10_isnull,id_13_isnull,addr1_target_code,addr1_fq_enc,C1_target_code,C1_fq_enc,C2_target_code,C2_fq_enc,C3_target_code,C3_fq_enc,C4_target_code,C4_fq_enc,C8_target_code,C8_fq_enc,C10_target_code,C10_fq_enc,C11_target_code,C11_fq_enc,C13_target_code,C13_fq_enc,C14_target_code,C14_fq_enc,card1_target_code,card1_fq_enc,card2_target_code,card2_fq_enc,card3_target_code,card4_target_code,card4_fq_enc,card5_target_code,D9_target_code,D9_fq_enc,device_name_target_code,device_name_fq_enc,DeviceType_target_code,DeviceType_fq_enc,id_12_target_code,id_12_fq_enc,id_13_target_code,id_13_fq_enc,id_14_target_code,id_14_fq_enc,id_15_target_code,id_15_fq_enc,id_16_target_code,id_16_fq_enc,id_17_target_code,id_17_fq_enc,id_19_target_code,id_19_fq_enc,id_20_target_code,id_23_target_code,id_26_target_code,id_28_target_code,id_29_target_code,id_30_device_target_code,id_30_device_fq_enc,id_30_version_target_code,id_30_version_fq_enc,id_32_target_code,id_32_fq_enc,id_33_target_code,id_33_fq_enc,id_33_0_target_code,id_33_0_fq_enc,id_33_1_target_code,id_33_1_fq_enc,id_33_area_target_code,id_33_area_fq_enc,id_33_ratio_target_code,id_33_ratio_fq_enc,id_35_target_code,id_35_fq_enc,id_36_target_code,id_37_target_code,id_37_fq_enc,id_38_target_code,R_emaildomain_target_code,R_emaildomain_fq_enc,cents_target_code,cents_fq_enc,DT_month,DT_week_year,DT_day_year,DT_hour,DT_day_week,DT_day,D1_day_mean,D1_week_mean,D2_day_mean,D2_week_mean,D3_day_mean,D3_week_mean,D6_day_mean,D6_week_mean,D7_day_mean,D7_week_mean,D8_day_mean,D8_week_mean,D10_day_mean,D10_week_mean,D13_day_mean,D13_week_mean,D14_day_mean,D14_week_mean,D15_day_mean,D15_week_mean
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [4]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.7,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':1000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 42,
                    'early_stopping_rounds':100, 
                } 

In [5]:
test_predictions, auc_score = make_predictions(train, test, train_y, features_columns, lgb_params)

Fold: 0  -  8578 3050
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[322]	training's auc: 1	valid_1's auc: 0.968254
Fold: 1  -  8639 2989
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[311]	training's auc: 1	valid_1's auc: 0.870905
Fold: 2  -  10119 1509
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[243]	training's auc: 1	valid_1's auc: 0.939699
Fold: 3  -  10242 1386
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[311]	training's auc: 1	valid_1's auc: 0.960855
Fold: 4  -  10257 1371
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[321]	training's auc: 1	valid_1's auc: 0.957144
Fold: 5  -  10305 1323
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[10]	training's auc: 0.995592	valid_

In [6]:
test_predictions.to_pickle('../input/mini-model-predictions/predictions_s.pkl')